<a href="https://colab.research.google.com/github/Minakshi85/Spam-Filter-for-Quora-Questions/blob/master/P3_Spam_filter_for_Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [3]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0

--2024-03-03 09:06:33--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2024-03-03 09:06:34--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7683c5bfb8183a1b473242af4b.dl.dropboxusercontent.com/cd/0/inline/COZmfbZ9jQVYQvNtattzn6LaAaJArfshFBfp2mkXlg-3XLg-Z9gp88Bx7gtHTK9cKwLI0X-5QFyY8H1J_dxg-KimKjNqoOUs-mITDI51S8hQjCDsmUNhhd41dwuDWR9AiBY/file# [following]
--2024-03-03 09:06:34--  https://uc7683c5bfb8183a1b473242af4b.dl.dropboxusercontent.com/cd/0/inline/COZmfbZ9jQVYQvNtattzn6LaAaJArfshFBfp2mkXlg-

In [4]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2024-03-03 09:06:43--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2024-03-03 09:06:43--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2024-03-03 09:06:44--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [5]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
replace glove.42B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
embedding_index ={}

f = open('/content/glove.42B.300d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype ='float32')
  embedding_index[word] = coefs
f.close()

In [8]:
question = pd.read_csv(r'/content/train.csv?dl=0')

In [9]:
print(question.shape)
question.head()

(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [10]:
X = question['question_text']
Y = question['target']
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1044897,), (1044897,), (261225,), (261225,))

In [11]:
y_test.value_counts()

0    245095
1     16130
Name: target, dtype: int64

In [12]:
#!pip install imbalanced-learn
import imblearn
from sklearn.utils.class_weight import compute_class_weight

In [13]:

class_weight = compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
#class_weight = imblearn.combine.compute_class_weight('balanced', np.unique(y_train), y_train)

#print(class_weight)
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5329926944747949, 1: 8.077435064935065}

In [14]:
x_train

1259130     How do I get rid of blood stains in my mattress?
521110     What does Dima Vorobiev think of the Russian g...
1084558              How hard is it to relocate as retirees?
99137      Which of the following documents are needed to...
1278471     How do I know if my dad doesn't love me anymore?
                                 ...                        
1166116    Does the fact of being black make you a criminal?
144095                         What is advantage of commers?
288747     In plain English, what is CRISPR? Why is it a ...
229735            Can the geography cone venom kill a human?
866051                 Hate keeps a man alive. Do you agree?
Name: question_text, Length: 1044897, dtype: object

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [17]:
print(np.percentile(sent_lens,99))
np.percentile(sent_lens,95)

45.0


31.0

In [18]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [19]:
x_train

1259130     How do I get rid of blood stains in my mattress?
521110     What does Dima Vorobiev think of the Russian g...
1084558              How hard is it to relocate as retirees?
99137      Which of the following documents are needed to...
1278471     How do I know if my dad doesn't love me anymore?
                                 ...                        
1166116    Does the fact of being black make you a criminal?
144095                         What is advantage of commers?
288747     In plain English, what is CRISPR? Why is it a ...
229735            Can the geography cone venom kill a human?
866051                 Hate keeps a man alive. Do you agree?
Name: question_text, Length: 1044897, dtype: object

In [20]:
seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)
print(len(seq_test), len(seq_train))
seq_train[0]

261225 1044897


[9, 11, 8, 32, 657, 7, 568, 6446, 6, 18, 8523]

In [21]:
vocab_size=len(tk.word_index)
vocab_size

196404

In [22]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 45), (261225, 45))

In [23]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index
# that we downloaded

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

# if there are specific words which are not present in pretrained embedding
# their weights will remain 0. if there are too many such words
# then you should probably not use pretrained embeddings


In [24]:
# Now while creating the model for text data , we'll keep embedding layer weights
# non-trainable as we are supplying those values from our end

inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [25]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 45)]              0         
                                                                 
 embedding (Embedding)       (None, 45, 300)           58921500  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [26]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True)

In [27]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [28]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5328573351855138, 1: 8.108651115146436},
              batch_size=32,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/10
 3732/32654 [==>...........................] - ETA: 7:11:24 - loss: 0.3527 - accuracy: 0.8492

In [ ]:
best_model = load_model('/content/weights-01-0.2338.h5')

In [ ]:
best_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 300)           58783200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [ ]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9528024281838164

In [ ]:
p

array([[0.39858234],
       [0.11947393],
       [0.6753924 ],
       ...,
       [0.01505166],
       [0.01708287],
       [0.29535776]], dtype=float32)